<a href="https://colab.research.google.com/github/talpt/pyton/blob/main/Kompozit_Banker_Dip_Sinyal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/rongardF/tvdatafeed  backtesting
!pip install tradingview-screener==2.5.0

import numpy as np
import pandas as pd
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
from backtesting import Backtest, Strategy
import warnings

warnings.simplefilter(action='ignore')


# Fonksiyonlar
def xsa(src, length, weight):
    sumf = np.zeros_like(src)
    ma = np.zeros_like(src)
    out = np.zeros_like(src)

    for i in range(length, len(src)):
        sumf[i] = np.nan_to_num(sumf[i - 1]) - np.nan_to_num(src[i - length]) + src[i]
        ma[i] = np.nan if np.isnan(src[i - length]) else sumf[i] / length
        out[i] = ma[i] if np.isnan(out[i - 1]) else (src[i] * weight + out[i - 1] * (length - weight)) / length

    return out

def ema(src, span):
    return src.ewm(span=span, adjust=False).mean()


def Banker_Fund_Trend(data):
    df = data.copy()
    close = df['close']
    low = df['low']
    high = df['high']
    open_ = df['open']
    fundtrend = np.zeros_like(close)
    part_1 = (close - low.rolling(window=27).min()) / (high.rolling(window=27).max() - low.rolling(window=27).min()) * 100
    fundtrend = ((3 * xsa(part_1, 5, 1) - 2 * xsa(xsa(part_1, 5, 1), 3, 1) - 50) * 1.032 + 50)

    typ = (2 * close + high + low + open_) / 5
    lol = low.rolling(window=34).min()
    hoh = high.rolling(window=34).max()

    bullbearline = ema((typ - lol) / (hoh - lol) * 100, 13)
    bankerentry = (fundtrend > bullbearline) & (bullbearline < 25)

    df['fundtrend'] = fundtrend
    df['bullbearline'] = bullbearline
    df['bankerentry'] = bankerentry.astype(int)

    return df


# Periyot Seçimi
PERIODS = {
    '1': ('Günlük', Interval.in_daily),
    '2': ('Haftalık', Interval.in_weekly),
    '3': ('Aylık', Interval.in_monthly)
}

print("Lütfen bir tarama periyodu seçin:")
for key, (desc, _) in PERIODS.items():
    print(f"{key}: {desc}")
period_choice = input("Seçiminiz: ")

if period_choice not in PERIODS:
    print("Geçersiz seçim, günlük periyot kullanılacak.")
    period_choice = '1'

selected_period, interval = PERIODS[period_choice]

# TvDatafeed ve Hisse Verileri
tv = TvDatafeed()
Hisseler = get_all_symbols(market='turkey')
Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
Hisseler = sorted(Hisseler)

# Raporlama
Titles = ['Hisse Adı', 'Son Fiyat', 'Dip Sinyali']
df_signals = pd.DataFrame(columns=Titles)

for hisse in Hisseler:
    try:
        data = tv.get_hist(symbol=hisse, exchange='BIST', interval=interval, n_bars=377)
        dataref = tv.get_hist(symbol='XU100', exchange='BIST', interval=interval, n_bars=377)
        data = data.reset_index()
        dataref = dataref.reset_index()
        result = data.copy()
        columns_to_divide = ['open', 'high', 'low', 'close', 'volume']
        for column in columns_to_divide:
            result[column] = 100 * (data[column] / dataref[column])
        Banker = Banker_Fund_Trend(result)
        Banker.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)
        Banker.set_index('datetime', inplace=True)

        Buy = False
        Signals = Banker.tail(2)
        Signals = Signals.reset_index()
        Buy = Signals.loc[0, 'bankerentry'] == 0 and Signals.loc[1, 'bankerentry'] == 1
        Last_Price = Signals.loc[1, 'Close']
        L1 = [hisse, round(Last_Price, 2), Buy]
        df_signals.loc[len(df_signals)] = L1
        print(L1)
    except Exception as e:
        print(f"{hisse} için hata: {e}")

df_True = df_signals[df_signals['Dip Sinyali'] == True]
print(df_True)


  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-36fbpy0q
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-36fbpy0q
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done
Lütfen bir tarama periyodu seçin:
1: Günlük
2: Haftalık
3: Aylık
Seçiminiz: 2


['A1CAP', 3.06, False]
['ACSEL', 1.17, False]
['ADEL', 0.35, False]
['ADESE', 0.02, False]
['ADGYO', 3.27, False]
['AEFES', 1.74, False]
['AFYON', 0.15, False]
['AGESA', 1.36, False]
['AGHOL', 3.07, False]
['AGROT', 1.17, False]
['AGYO', 0.08, False]
['AHGAZ', 2.0, False]
['AHSGY', 1.75, False]
['AKBNK', 0.64, False]
['AKCNS', 1.8, False]
['AKENR', 0.14, False]
['AKFGY', 0.02, False]
['AKFYE', 2.09, False]
['AKGRT', 0.07, False]
['AKMGY', 2.24, False]
['AKSA', 0.12, False]
['AKSEN', 0.41, False]
['AKSGY', 0.08, False]
['AKSUE', 0.12, False]
['AKYHO', 0.03, False]
['ALARK', 0.95, False]
['ALBRK', 0.07, False]
['ALCAR', 10.37, False]
['ALCTL', 1.26, False]
['ALFAS', 6.37, False]
['ALGYO', 0.21, False]
['ALKA', 0.09, False]
['ALKIM', 0.19, False]
['ALKLC', 3.59, False]
['ALMAD', 0.08, False]
['ALTIN', 2.79, False]
['ALTNY', 8.72, False]
['ALVES', 4.07, False]
['ANELE', 0.16, False]
['ANGEN', 1.01, False]
['ANHYT', 0.98, False]
['ANSGR', 1.01, False]
['APBDL', 2.19, False]
['APX30', 2.57, 

ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


KERVN için hata: 'NoneType' object has no attribute 'reset_index'
['KERVT', 0.15, False]
['KFEIN', 0.83, False]
['KGYO', 0.5, False]
['KIMMR', 0.69, True]
['KLGYO', 0.05, False]
['KLKIM', 2.38, False]
['KLMSN', 0.3, False]
['KLNMA', 0.14, False]
['KLRHO', 3.04, False]
['KLSER', 3.81, False]
['KLSYN', 0.57, False]
['KMPUR', 1.71, False]
['KNFRT', 0.12, False]
['KOCMT', 1.73, False]
['KONKA', 2.97, False]
['KONTR', 1.87, False]
['KONYA', 66.44, False]
['KOPOL', 0.6, False]
['KORDS', 0.74, False]
['KOTON', 2.12, False]
['KOZAA', 0.73, False]
['KOZAL', 0.23, False]
['KRDMA', 0.3, False]
['KRDMB', 0.25, False]
['KRDMD', 0.28, False]
['KRGYO', 0.08, False]
['KRONT', 0.22, False]
['KRPLS', 0.82, False]
['KRSTL', 0.06, False]
['KRTEK', 0.41, False]
['KRVGD', 0.12, False]
['KSTUR', 29.21, False]
['KTLEV', 6.74, False]
['KTSKR', 4.56, False]
['KUTPO', 0.81, False]
['KUVVA', 0.66, False]
['KUYAS', 0.21, False]
['KZBGY', 0.52, False]
['KZGYO', 2.22, False]
['LIDER', 12.41, False]
['LIDFA', 0.03, T